<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">KSO Tutorials #6: Evaluate machine learning models</h1>
<h3 align="right"><a href="https://colab.research.google.com/github/ocean-data-factory-sweden/kso/blob/master/tutorials/06_Evaluate_ML_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a></h3>
<h3 align="right">Written by the KSO Team</h3>

# Set up KSO requirements

### Install all the requirements

If you are running this notebook in Google Colab, this cell should take ~2 mins and might restart the session. Please run this cell until you get the "Successful installation!" message.

In [1]:
import sys
import os

# Check if notebook is running in colab
IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
    # Clone kso repo and install requirements
    if not os.path.exists("kso"):
        print("Installing all dependencies...")
        !git clone https://github.com/ocean-data-factory-sweden/kso.git
        !pip install -r /content/kso/requirements_colab.txt

    # Enable external widgets and navigate to the kso tutorial folder
    try:
        from google.colab import output

        output.enable_custom_widget_manager()
        os.chdir("kso/tutorials")
    except ImportError:
        pass

# Prepare the dev settings if needed
try:
    if "kso_utils" not in sys.modules:
        sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))
        import kso_utils

        print("Using development version...")
        # Enables testing changes in utils
        %load_ext autoreload
        %autoreload 2
except ImportError:
    print("Installing latest version from PyPI...")
    %pip install -q kso-utils

if IN_COLAB:

    def restart_runtime():
        os.kill(os.getpid(), 9)

    # Check if there are any issues with previously imported packages
    try:
        from kso_utils.project import ProjectProcessor
    except Exception as e:
        print(f"Error importing package: {e}")
        print("Restarting runtime to apply package changes...")
        restart_runtime()

# Avoid issues with widgets not displaying properly
!jupyter nbextension enable --user --py widgetsnbextension
!jupyter nbextension enable --user --py jupyter_bbox_widget
!jupyter nbextension enable --user --py ipysheet

# Load the clear output function to keep things clean
from IPython.display import clear_output

clear_output()
print("Successful installation... you're good to go!")

Successful installation... you're good to go!


### Import python packages

In [2]:
# Import required modules for tut#6
import kso_utils.widgets as kso_widgets
import kso_utils.project_utils as p_utils
import kso_utils.tutorials_utils as t_utils
import kso_utils.yolo_utils as y_utils
from kso_utils.project import ProjectProcessor, MLProjectProcessor
import os

print("Packages loaded successfully")

Packages loaded successfully


### Choose your project

In [3]:
project_name = kso_widgets.choose_project()

Dropdown(description='Project:', options=('Template project', 'Koster_Seafloor_Obs', 'Spyfish_Aotearoa', 'SGU'…

### Initiate project's database

In [4]:
# Find project
project = p_utils.find_project(project_name=project_name.value)
# Initialise pp
pp = ProjectProcessor(project)

INFO:root:Template project loaded succesfully
INFO:root:Retrieving the file from https://drive.google.com/uc?&confirm=s5vl&id=1PZGRoSY_UpyLfMhRphMUMwDXw4yx1_Fn
Downloading...
From: https://drive.google.com/uc?&confirm=s5vl&id=1PZGRoSY_UpyLfMhRphMUMwDXw4yx1_Fn
To: /Users/jurie.germishuys/Workspace/odf/koster-uw/kso/tutorials/db_csv_info.zip
100%|██████████| 3.23k/3.23k [00:00<00:00, 2.89MB/s]
INFO:root:Updated species table from the temporary database
INFO:root:Updated sites table from the temporary database
INFO:root:Updated movies table from the temporary database


In [5]:
# Initialise mlp
mlp = MLProjectProcessor(pp)

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jannesg (koster). Use `wandb login --relogin` to force relogin


# Evaluate model on custom footage

### Choose the model

In [6]:
model = mlp.choose_model()

Dropdown(description='Select model:', layout=Layout(width='50%'), options=(('snic_yolov8_test', 'run_snaegr15_…

Output()

### Choose folder to download the model

In [ ]:
download_dir = kso_widgets.choose_folder(".", "where to download the model")

### Download model

In [ ]:
artifact_dir = mlp.get_model(model.value, download_dir.selected)

### Choose the footage to run the models into

In [ ]:
pp.choose_footage(preview_media=True)

Uploaded Footage selected


SelectMultiple(description='Select movie(s):', layout=Layout(width='50%'), options=('movie_1.mp4', 'movie_2.mp…

Output()

Output()

ERROR:asyncio:Task was destroyed but it is pending!
task: <Task pending name='Task-10' coro=<ProjectProcessor.choose_footage.<locals>.on_radio_button_change.<locals>.f() done, defined at /Users/jurie.germishuys/Workspace/odf/koster-uw/kso/kso_utils/project.py:309> wait_for=<Future pending cb=[Task.task_wakeup()]>>
ERROR:asyncio:Task was destroyed but it is pending!
task: <Task pending name='Task-11' coro=<ProjectProcessor.choose_footage.<locals>.on_radio_button_change.<locals>.f() done, defined at /Users/jurie.germishuys/Workspace/odf/koster-uw/kso/kso_utils/project.py:309> wait_for=<Future pending cb=[Task.task_wakeup()]>>
ERROR:asyncio:Task was destroyed but it is pending!
task: <Task pending name='Task-12' coro=<ProjectProcessor.choose_footage.<locals>.on_radio_button_change.<locals>.f() done, defined at /Users/jurie.germishuys/Workspace/odf/koster-uw/kso/kso_utils/project.py:309> wait_for=<Future pending cb=[Task.task_wakeup()]>>


### Choose a confidence threshold for evaluation

In [ ]:
conf_thres = t_utils.choose_conf()

## Choose a suitable experiment name

In [ ]:
exp_name = t_utils.choose_experiment_name()

### Choose a suitable folder to store model output

In [ ]:
save_dir = kso_widgets.choose_folder(".", "where to store model output")

### Run model over selected custom footage

In [ ]:
mlp.detect_yolo(
    source=pp.movies_paths,
    save_dir=save_dir.selected,
    conf_thres=conf_thres.value,
    artifact_dir=artifact_dir,
    save_output=True,
    project=mlp.project_name,
    name=exp_name.value,
    model=model.value,
)

### View model output

In [ ]:
viewer = y_utils.choose_files(mlp.eval_dir)

### Investigate training and validation datasets (only image data)

In [ ]:
train_dataset, val_dataset = mlp.get_dataset(model.value)

In [ ]:
# Training set
y_utils.get_data_viewer(os.path.join(train_dataset, "data/images"))

In [ ]:
# Validation set
y_utils.get_data_viewer(os.path.join(val_dataset, "data/images"))

# Track unique individuals (Optional)

In [ ]:
mlp.track_individuals(
    name=mlp.project_name,
    source=pp.movies_paths,
    artifact_dir=artifact_dir,
    conf_thres=conf_thres.value,
    img_size=(640, 640),
)

# (Optional) Clean noisy tracking output

In [ ]:
# Choose another evaluation directory
mlp.eval_dir = kso_widgets.choose_folder(".", "tracking folder")

In [ ]:
y_utils.adjust_tracking(
    tracking_folder=mlp.eval_dir.selected,
    avg_diff_frames=90,
    min_frames_length=90,
    plot_result=True,
)

In [ ]:
# END